In [73]:
import numpy as np
from scipy.stats import kurtosis, mode, skew
from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression   

In [23]:
# Para leer el tipo de archivo .data se utiliza la siguiente función
with open('synthetic_control.data') as input_file:
    lines = input_file.readlines()
    newLines = []
    for line in lines:
        newLine = line.strip().split()
        newLines.append(newLine )
dato = newLines
# devuelve una lista de tipo string
# lo transformamos por numeros
datos = np.array(dato).astype(float)

In [12]:
# Definimos las 7 caracteristicas para los datos
def caracteristicas(X):
    M = []
    for i in range(len(X)):
        # Guardamos las 7 caracteristicas para cada observación
        M.append([np.mean(X[i,:]),np.median(X[i,:]),kurtosis(X[i,:]),np.std(X[i,:]),np.var(X[i,:]),skew(X[i,:]),mode(X[i,:])[0][0]])
    return M

In [41]:
# Transformamos las caracteristicas en 
X = np.array(caracteristicas(datos))

In [59]:
# Preprocesamos los datos
def preprocesamiento(X_train,X_test):
    #Remoción para x_train
    def Remocion(X_train):
        med = X_train.mean(axis=0)
        des = X_train.std(axis=0)
        X_train = X_train - med
        X_train = X_train/des
        return X_train,med, des

    #Remocion para x_test
    def Remocion_test(X_test,med,des):
        X_test = X_test - med
        X_test = X_test/des
        return X_test
    #nomralizacion de x_train
    x_norm_train,t_train_mean,t_train_desv = Remocion(X_train)
    #Normalizacion para x_test
    x_norm_test = Remocion_test(X_test,t_train_mean,t_train_desv)
    
    return x_norm_test,x_norm_train


In [405]:
# Definimos Y
y = []
y = [1]*100 + [2]*100 + [3]*100 + [4]*100 + [5]*100 + [6]*100 

In [406]:
# Selector de caracteristicas solo 4
def selectores(X_test,X_train,y_train):
    selector = SelectKBest(f_regression, k=4)
    selector.fit(X_train,y_train)
    X_test = selector.transform(X_test)
    X_train = selector.transform(X_train)
    return(X_test,X_train)

In [407]:
# Clasificador de caracteristicas
def clasificadores(X_train, y_train, X_test):
    # Entrenamos el clasificador
    clasificador = LogisticRegression(C=100.0) # C es el parametro
    clasificador.fit(X_train, y_train) # entrenamiento del clasificador
    # Para validar el clasificador
    y_pred = clasificador.predict(X_test)
    acc = 100.0*(y_pred == y_test).sum()/X_test.shape[0]
    return acc

In [429]:
sum_data_acc = []
for i in range(10):
    # Dividimos los datos
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.3)
    # Preprocesamos
    X_test,X_train = preprocesamiento(X_train,X_test)
    # Seleccionamos algunas caracteristicas( En nuestro caso 4)
    X_test,X_train = selectores(X_test,X_train,y_train)
    # Guardamos los resultados de los aciertos
    sum_data_acc.append(clasificadores(X_train, y_train, X_test))
# Mostramos el resultado
print('El resultado de acciertos es de : %f, Con un error de +/- %f'%(np.mean(sum_data_acc),np.std(sum_data_acc)))


El resultado de acciertos es de : 89.888889, Con un error de +/- 4.673276
